# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [4]:
from pandas import *

In [15]:
url = ("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv")

In [31]:
df = read_csv(url)
df.columns = df.columns.str.replace(" ","_")
df.columns = df.columns.str.lower()

In [79]:
df.columns

Index(['unnamed:_0', 'customer', 'state', 'customer_lifetime_value',
       'response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'gender', 'income', 'location_code',
       'marital_status', 'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type'],
      dtype='object')

In [81]:
df = df[(df["total_claim_amount"] > 1000) & (df["response"] == "Yes")]

In [88]:
df.groupby(["policy_type", "gender"])["total_claim_amount"].mean()

policy_type     gender
Corporate Auto  F         1138.400000
                M         1171.150007
Personal Auto   F         1214.853805
                M         1137.861443
Special Auto    F         1358.400000
                M         1017.500015
Name: total_claim_amount, dtype: float64

In [98]:
df.groupby(["education", "gender"])["total_claim_amount"].describe()

count         mean         std          min  \
education            gender                                                
Bachelor             F        20.0  1226.880000  148.627948  1032.000000   
College              F         7.0  1027.200000    0.000000  1027.200000   
                     M        14.0  1092.139400   87.315582  1008.000000   
High School or Below M        14.0  1175.900014  154.520818  1027.000029   
Master               F        12.0  1278.010146   17.432426  1261.319869   

                                     25%          50%          75%  \
education            gender                                          
Bachelor             F       1032.000000  1300.800000  1358.400000   
College              F       1027.200000  1027.200000  1027.200000   
                     M       1008.000000  1092.139400  1176.278800   
High School or Below M       1027.000029  1175.900014  1324.800000   
Master               F       1261.319869  1278.010146  1294.700423   

                                     max  
education            gender               
Bachelor             F       1358.400000  
College              F       1027.200000  
                     M       1176.278800  
High School or Below M       1324.800000  
Master               F       1294.700423

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here